In [1]:
import tensorflow as tf

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
def routing(input, b):
    W = tf.get_variable('W', shape=(1, 1152, 10, 8, 16), dtype=tf.float32)
    input = tf.tile(input, [1, 1, 10, 1, 1])
    W = tf.tile(W, [batch_size, 1, 1, 1, 1])
    u_hat = tf.matmul(W, input, transpose_a=True)

    for iter in range(iterations):
        c = tf.nn.softmax(b, dim=2)
        c = tf.tile(c, [batch_size, 1, 1, 1, 1])
        s = tf.multiply(c, u_hat)
        s = tf.reduce_sum(s, axis=1, keep_dims=True)
        v = squash(s)
        v_tiled = tf.tile(v, [1, 1152, 1, 1, 1])
        u_produce_v = tf.matmul(u_hat, v_tiled, transpose_a=True)
        b += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)

    return(v)

In [ ]:
with tf.variable_scope('conv1'):
    conv1 = tf.contrib.layers.conv2d(X, num_outputs=256, kernel_size=9, stride=1, padding='VALID')
with tf.variable_scope('primary_capsule'):
    primaryCaps = CapsLayer(num_outputs=32, vec_len=8, with_routing=False, layer_type='CONV')
    caps1 = primaryCaps(conv1, kernel_size=9, stride=2)
with tf.variable_scope('digit_capsule'):
    digitCaps = CapsLayer(num_outputs=10, vec_len=16, with_routing=True, layer_type='FC')
    caps2 = digitCaps(caps1)
with tf.variable_scope('masking'):
    v_length = tf.sqrt(tf.reduce_sum(tf.square(caps2), axis=2, keep_dims=True) + epsilon)
    softmax_v = tf.nn.softmax(v_length, dim=1)
    argmax_idx = tf.to_int32(tf.argmax(softmax_v, axis=1))
    argmax_idx = tf.reshape(argmax_idx, shape=(batch_size, ))
    if not mask_with_y:
        masked_v = []
        for batch_size in range(batch_size):
            v = caps2[batch_size][argmax_idx[batch_size], :]
            masked_v.append(tf.reshape(v, shape=(1, 1, 16, 1)))
            masked_v = tf.concat(masked_v, axis=0)
    else:
        masked_v = tf.multiply(tf.squeeze(caps2), tf.reshape(Y, (-1, 10, 1)))
        v_length = tf.sqrt(tf.reduce_sum(tf.square(caps2), axis=2, keep_dims=True) + epsilon)


In [ ]:
vector_j = tf.reshape(masked_v, shape=(batch_size, -1))
fc1 = tf.contrib.layers.fully_connected(vector_j, num_outputs=512)
fc2 = tf.contrib.layers.fully_connected(fc1, num_outputs=1024)
decoded = tf.contrib.layers.fully_connected(fc2, num_outputs=784, activation_fn=tf.sigmoid)    